In [77]:
import pandas as pd
from datetime import datetime, timedelta
import math

In [112]:
#the players number column, age, weight columns have NaN's in them probably can drop the number and weight columns
#as they are not features 
roster = pd.read_csv('stats_files/rosters', sep = '|')
roster.shape


/Users/MattBarlowe/.virtualenvs/prospectmodel/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,2,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(136182, 13)

In [113]:
#makes sure age matches this format and removes players which we don't have a birthdate for
#also excludes weird dates that might include dates over 31 or months over 12 or entries of 00
roster = roster[roster['Birthdate'].str.match('(\d\d\d\d)\-((?!00)0[1-9]|1[0-2])\-((?!00)(0[1-9]|[12]\d|3[01]))')]
roster.shape

(136021, 13)

In [114]:
#had two birthdates on september 31st just excluded them 
roster = roster[~roster['Birthdate'].str.match('(\d\d\d\d)\-09\-31')]
roster.shape

(136018, 13)

In [115]:
#Create the season start to calculate age at start of season
roster['season_start'] = roster['season'].apply(lambda x: x - 1).apply('09-15-{}'.format)
roster['Birthdate'] = pd.to_datetime(roster['Birthdate'], format='%Y-%m-%d')
roster['season_start'] = pd.to_datetime(roster['season_start'], format='%m-%d-%Y')
roster['draft_cutoff'] = roster['season'].apply('09-15-{}'.format)
roster['draft_cutoff'] = pd.to_datetime(roster['draft_cutoff'], format='%m-%d-%Y')
#EP lists players Age as their current age instead of the age they were at the season
#I'm calculating their age as the age they are at the draft cutoff of that season
#in order to see if they are draft eligible that season
roster['Age'] = (roster['season_start'] - roster['Birthdate'])
roster['Age'] = roster['Age'] / timedelta(days=365)
roster['Age'] = roster['Age'].round(decimals=2)
roster['draft_cutoff_age'] = roster['draft_cutoff'] - roster['Birthdate']
roster['draft_cutoff_age'] = roster['draft_cutoff_age'] / timedelta(days=365)
roster['draft_cutoff_age'] = roster['draft_cutoff_age'].round(decimals=2)

In [131]:
def draft_year(age):
    
    if age >=18 and age < 19:
        return 0
    else:
        return math.floor(age-18)
        
        
roster['draft_year'] = roster['draft_cutoff_age'].apply(draft_year)
#roster.Age.astype

In [132]:
roster[['Player','team','Age','draft_year', 'draft_cutoff_age','season']].tail()


,Player,team,Age,draft_year,draft_cutoff_age,season
136177,Ryan Peckford,Victoria Royals,18.55,1,19.55,2018
136178,Matthew Phillips,Victoria Royals,19.46,2,20.46,2018
136179,Tyler Soy,Victoria Royals,20.61,3,21.61,2018
136180,Ty Yoder,Victoria Royals,15.25,-2,16.25,2018
136181,Lane Zablocki,Victoria Royals,18.73,1,19.73,2018


In [133]:
roster[(roster.team=='Victoria Royals') & (roster.season==2016)].Player

134577     Griffen Outhouse 
134578     Coleman Vollrath 
134579          Ryan Gagnon 
134580         Joe Hicketts 
134581     Marsel Ibragimov 
134582        Ralph Jarratt 
134583       Brayden Pachal 
134584       Chaz Reddekopp 
134585        Scott Walford 
134586       Jordan Wharrie 
134587       Keith Anderson 
134588    Vladimir Bobylyov 
134589        Jared Dmytriw 
134590         Matt Dykstra 
134591         Logan Fisher 
134592        Alex Forsberg 
134593        Dante Hannoun 
134594           Regan Nagy 
134595        Ryan Peckford 
134596     Matthew Phillips 
134597          Ethan Price 
134598            Tyler Soy 
134599       Tyler Thompson 
134600          Jack Walker 
134601          Ty Westgard 
Name: Player, dtype: object